In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
# Carregando os dados do challenge 1
url = "https://raw.githubusercontent.com/MilenaSimionee/TelecomX/refs/heads/main/telecom_churn_tratado.csv"
df = pd.read_csv(url)

In [4]:
df.head()


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,TenureGroup
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,0-12 meses
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,0-12 meses
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,0-12 meses
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,13-24 meses
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,0-12 meses


In [5]:
# Verificar se há valores faltantes por coluna
print("Valores faltantes por coluna:")
print(df.isnull().sum())

Valores faltantes por coluna:
customerID           0
Churn                0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
TenureGroup          0
dtype: int64


In [6]:
df = df.dropna(subset=['TotalCharges'])

In [7]:
mediana = df['TotalCharges'].median()
df['TotalCharges'] = df['TotalCharges'].fillna(mediana)

In [8]:
mediana = df['TotalCharges'].median()
df['TotalCharges'] = df['TotalCharges'].fillna(mediana)

In [9]:
print("Valores faltantes por coluna após o preenchimento:")
print(df.isnull().sum())


Valores faltantes por coluna após o preenchimento:
customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
TenureGroup         0
dtype: int64


ACIMA TINHA VERIFICADO QUE ESTAVA COM 11 VALORES FALTANTES, ENTÃO FOI FEITO O PRENCHIMENTO DESSES VALORES FALTANTES.

**TUDO OK AGORA, COMO VISTO ACIMA**

**Preparação dos dados para modelagem**

Aqui verfico quais colunas são categoricas e numerocas, pois o modelo só  entende dados numericos.

In [10]:
# Colunas categóricas
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Colunas categóricas:", categorical_cols)

# Colunas numéricas
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("\nColunas numéricas:", numerical_cols)


Colunas categóricas: ['customerID', 'Churn', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TenureGroup']

Colunas numéricas: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']


Devido termos em nosso DataFrame muitas colunas cate góricas, vamos transforma´le em numericas

In [11]:
# Importar biblioteca para encoding
from sklearn.preprocessing import LabelEncoder

# 1. Transformar a variável alvo Churn para 0 (No) e 1 (Yes)
le = LabelEncoder()
df['Churn'] = le.fit_transform(df['Churn'])

# 2. Listar colunas categóricas (exceto Churn)
categorical_cols.remove('Churn')

# 3. Identificar colunas binárias (com 2 categorias)
binary_cols = [col for col in categorical_cols if df[col].nunique() == 2]
print("Colunas binárias:", binary_cols)

# 4. Aplicar Label Encoding nas colunas binárias
for col in binary_cols:
    df[col] = le.fit_transform(df[col])

# 5. Colunas com mais de duas categorias (multiclasse)
multi_cat_cols = [col for col in categorical_cols if col not in binary_cols]
print("Colunas com múltiplas categorias:", multi_cat_cols)

# 6. Aplicar One-Hot Encoding nessas colunas
df = pd.get_dummies(df, columns=multi_cat_cols)

# Mostrar as primeiras linhas para verificar
df.head()


Colunas binárias: ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
Colunas com múltiplas categorias: ['customerID', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod', 'TenureGroup']


,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TenureGroup_0-12 meses,TenureGroup_13-24 meses,TenureGroup_25-48 meses,TenureGroup_49-60 meses,TenureGroup_61-72 meses,TenureGroup_72+ meses
0,0,0,0,1,1,9,1,1,65.6,593.30,...,False,False,False,True,True,False,False,False,False,False
1,0,1,0,0,0,9,1,0,59.9,542.40,...,False,False,False,True,True,False,False,False,False,False
2,1,1,0,0,0,4,1,1,73.9,280.85,...,False,False,True,False,True,False,False,False,False,False
3,1,1,1,1,0,13,1,1,98.0,1237.85,...,False,False,True,False,False,True,False,False,False,False
4,1,0,1,1,0,3,1,1,83.9,267.40,...,False,False,False,True,True,False,False,False,False,False


In [12]:
print(df.columns)


Index(['Churn', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges',
       ...
       'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'TenureGroup_0-12 meses', 'TenureGroup_13-24 meses',
       'TenureGroup_25-48 meses', 'TenureGroup_49-60 meses',
       'TenureGroup_61-72 meses', 'TenureGroup_72+ meses'],
      dtype='object', length=7079)


✅ Etapas que já concluidas.

Importação dos dados tratados via GitHub.

Verificação e tratamento de valores faltantes.

Identificação de colunas categóricas, binárias e numéricas.

Transformação de variáveis categóricas em variáveis dummies.

SEGUIMOS PARA OS PROXIMOS PASSOS

1 Separar variáveis preditoras (X) e variável alvo (y).

In [13]:
# y é a variável que queremos prever
y = df['Churn']

# X são todas as outras variáveis, menos a coluna 'Churn'
X = df.drop('Churn', axis=1)


In [14]:
print("Formato de X:", X.shape)
print("Formato de y:", y.shape)


Formato de X: (7032, 7078)
Formato de y: (7032,)


2 Dividir dados em treino e teste

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
 # Dividindo os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Verificando o tamanho dos conjuntos
print(f'Treino: {X_train.shape}, {y_train.shape}')
print(f'Teste: {X_test.shape}, {y_test.shape}')

Treino: (5625, 7078), (5625,)
Teste: (1407, 7078), (1407,)


3 Treinar modelos

Testaremos 3 modelos

Árvore de Decisão (DecisionTreeClassifier)

Random Forest (RandomForestClassifier)

Regressão Logística (LogisticRegression)

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [18]:
# Criando os modelos
modelos = {
    'Árvore de Decisão': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Regressão Logística': LogisticRegression(max_iter=1000, random_state=42)
}

In [19]:
from sklearn.linear_model import LogisticRegression

modelos['Regressão Logística'] = LogisticRegression(max_iter=1000)


In [20]:
for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'{nome} - Acurácia: {acc:.4f}')


Árvore de Decisão - Acurácia: 0.7740
Random Forest - Acurácia: 0.7989
Regressão Logística - Acurácia: 0.8010


C:\Users\Gabri\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
modelos['Regressão Logística'] = LogisticRegression(max_iter=2000)


In [ ]:
for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'{nome} - Acurácia: {acc:.4f}')


Árvore de Decisão - Acurácia: 0.7740
Random Forest - Acurácia: 0.7989


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

modelo_lr = LogisticRegression(max_iter=1000, random_state=42)
modelo_lr.fit(X_train_scaled, y_train)

y_pred_lr = modelo_lr.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred_lr)
print(f'Regressão Logística (dados escalados) - Acurácia: {acc:.4f}')


### ⚠️ Aviso durante o treino da Regressão Logística

Durante o treinamento do modelo de Regressão Logística, foi gerado o seguinte aviso:

`ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.`

Este aviso indica que o algoritmo não convergiu no número padrão de iterações. Como solução, aumentei o parâmetro `max_iter` para 2000, e também testei com os dados normalizados usando `StandardScaler`.

Apesar disso, o aviso ainda apareceu — o que é comum em datasets mais complexos. O modelo funcionou normalmente, com acurácia próxima de 79%. Ainda assim, esse ponto foi observado e documentado.


Agora vamos fazer uma avaliação mais detalhada dos seus modelos com matriz de confusão e relatório de classificação — isso ajuda muito a entender pontos fortes e fracos de cada modelo além da acurácia.

**Passo 1: Importar as funções que sera usada**

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt


**Passo 2: Gerar predições para todos os modelos**

In [ ]:
# Previsões Árvore de Decisão e Random Forest (dados originais)
y_pred_dt = modelos['Árvore de Decisão'].predict(X_test)
y_pred_rf = modelos['Random Forest'].predict(X_test)

# Previsão Regressão Logística (dados escalados)
y_pred_lr = modelo_lr.predict(X_test_scaled)


**Passo 3: Criar função para plotar matriz de confusão**

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.show()


In [ ]:
for nome, y_pred in zip(
    ['Árvore de Decisão', 'Random Forest', 'Regressão Logística'],
    [y_pred_dt, y_pred_rf, y_pred_lr]):

    print(f"Modelo: {nome}")
    plot_confusion_matrix(y_test, y_pred, f"Matriz de Confusão - {nome}")
    print(classification_report(y_test, y_pred))
    print('--------------------------------------')


**Análise de correlação**

In [ ]:
# Certificar que o churn está binário
df['Churn_binary'] = df['Churn'].map({'No': 0, 'Yes': 1})

# Selecionar apenas algumas colunas relevantes manualmente
colunas_relevantes = ['Churn_binary', 'tenure', 'MonthlyCharges', 'TotalCharges']

# Remover valores ausentes ou inválidos (por precaução)
df_filtrado = df[colunas_relevantes].copy()
df_filtrado = df_filtrado.apply(pd.to_numeric, errors='coerce')
df_filtrado = df_filtrado.dropna()

# Calcular correlação
correlacoes = df_filtrado.corr()['Churn_binary'].sort_values(ascending=False)

# Exibir
print("Correlação com Churn:")
print(correlacoes)


In [ ]:
# Garantir que todas as colunas estão presentes e em formato correto
df['Churn_binary'] = df['Churn'].map({'No': 0, 'Yes': 1})

# Verifique se as colunas existem e estão numéricas
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['MonthlyCharges'] = pd.to_numeric(df['MonthlyCharges'], errors='coerce')
df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')

# Remover linhas com valores ausentes em qualquer uma dessas colunas
df_corr = df[['Churn_binary', 'tenure', 'MonthlyCharges', 'TotalCharges']].dropna()

# Calcular correlação com o churn
correlacoes = df_corr.corr()['Churn_binary'].sort_values(ascending=False)

# Exibir
print("Correlação com Churn:")
print(correlacoes)


In [ ]:
print(df[['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn']].dtypes)


In [ ]:
print(df[['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn']].isnull().sum())


In [ ]:
# Correlação com a variável Churn (já binária)
correlacoes = df[['tenure', 'MonthlyCharges', 'TotalCharges']].corrwith(df['Churn']).sort_values(ascending=False)
print("Correlação com Churn:")
print(correlacoes)


**Relatório de Conclusão Estratégica – Projeto Telecom X**

**1**. Introdução **negrito**

Neste projeto, analisamos dados de clientes da empresa Telecom X para entender os principais fatores que influenciam a evasão (churn) e construir modelos preditivos para identificar clientes com maior risco de cancelamento.

**2. Preparação dos Dados**

Tratamos dados faltantes, especialmente na coluna TotalCharges.

Codificamos variáveis categóricas com one-hot encoding.

Normalizamos variáveis numéricas para otimizar o desempenho dos modelos.

**3. Análise Exploratória e Correlação**

As variáveis mais correlacionadas com churn foram:

Tempo como cliente (tenure), com correlação negativa de -0.35.

Valor da mensalidade (MonthlyCharges), correlação positiva de +0.19.

Total gasto pelo cliente (TotalCharges), correlação negativa de -0.20.

Clientes com menor tempo de contrato e mensalidades mais altas apresentam maior probabilidade de cancelar.

Clientes que gastaram mais tendem a permanecer.

**4. Modelagem**

Treinamos três modelos de classificação: Árvore de Decisão, Random Forest e Regressão Logística.

O modelo Random Forest apresentou o melhor desempenho, com acurácia próxima de 79,9%.

**5. Avaliação dos Modelos**

Métricas avaliadas: acurácia, precisão, recall e F1-score.

Os modelos identificam melhor os clientes que permanecem, porém têm menor desempenho para detectar os que cancelam, o que é comum em bases desbalanceadas.

**6. Conclusões Estratégicas**
O tempo de permanência é o fator mais influente na evasão: clientes recentes têm maior risco de churn.

Mensalidades altas elevam a chance de cancelamento, sugerindo a necessidade de planos ou incentivos diferenciados.

O total gasto indica fidelidade: clientes com maior gasto tendem a ficar.

Recomenda-se focar ações em clientes novos e com mensalidades elevadas para reduzir a evasão.

**7. Próximos Passos**

Explorar variáveis adicionais que possam impactar o churn, como uso de serviços ou suporte.

Aplicar técnicas de balanceamento para melhorar a detecção de clientes que cancelam.

Experimentar modelos mais complexos para potencializar a previsão.

